# Creating the Data Base

In [155]:
pip install -U scikit-learn


Requirement already up-to-date: scikit-learn in /opt/anaconda3/lib/python3.8/site-packages (0.24.1)
Note: you may need to restart the kernel to use updated packages.


In [156]:
pip install glob2 

Note: you may need to restart the kernel to use updated packages.


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import numpy as np
import pickle
from tqdm import  tqdm
import shutil

pd.options.mode.chained_assignment = None
pd.options.display.min_rows = 50

# Scraping the data

In [21]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import socket
import os
import pickle

Basketball-refrence.com scraper:

In [66]:
class BbrefScraper:
    def __init__(self, season_start_links, scrape_type):
        self.scrape_type = scrape_type
        self.season_start_links = season_start_links
        self.base_url = 'https://www.basketball-reference.com'
        #change to your own directoty
        self.data_path = '/Users/yoavhaim/Desktop/Programming/project'
        if not os.path.exists(self.data_path):
            os.mkdir(self.data_path)
            os.mkdir(f'{self.data_path}/pickles')
            os.mkdir(f'{self.data_path}/bbref-files')

        self.team_dictionary = {
            'Atlanta Hawks': 'Atl', 'Boston Celtics': 'Bos', 'Brooklyn Nets': 'Bkn', 'Charlotte Hornets': 'Cha',
            'Chicago Bulls': 'Chi', 'Cleveland Cavaliers': 'Cle', 'Dallas Mavericks': 'Dal', 'Denver Nuggets': 'Den',
            'Detroit Pistons': 'Det', 'Golden State Warriors': 'GSW', 'Houston Rockets': 'Hou', 'Indiana Pacers': 'Ind',
            'Los Angeles Lakers': 'LAL', 'Los Angeles Clippers': 'LAC', 'Memphis Grizzlies': 'Mem', 'Miami Heat': 'Mia',
            'Milwaukee Bucks': 'Mil', 'Minnesota Timberwolves': 'Min', 'New Orleans Pelicans': 'Nor', 'New York Knicks': 'NYK',
            'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'Orl', 'Philadelphia 76ers': 'Phi', 'Phoenix Suns': 'Pho',
            'Portland Trail Blazers': 'Por', 'Sacramento Kings': 'Sac', 'San Antonio Spurs': 'SAS', 'Toronto Raptors': 'Tor',
            'Utah Jazz': 'Uta', 'Washington Wizards': 'Was'
        }

    def scrape_stats(self):
        print(f'Scraping\n','~' * 50, '\n', '\n'.join(self.season_start_links))
        #get the url for each month in the season
        self.get_months()
        # print(self.month_url_dict)

        #get the game links for each game within each month
        self.get_game_links()
        # print(self.full_game_urls)

        #scrape each table in each game and save to dataframe
        self.get_game_stats()

    def get_game_stats(self):
        # self.full_game_urls = pickle.load(open(f'{self.data_path}/pickles/GameLinks.p', 'rb'))
        for season, game_links in self.full_game_urls.items():
            pieces = []
            pbar = tqdm(game_links, desc = f'Scraping Games: {season}')
            # total = 0
            for link in pbar:
                response = requests.get(link)
                soup = BeautifulSoup(response.text, 'html.parser')
                table_df = self.get_table_info(soup, link)
                pieces.append(table_df)
                # total += 1
                # if total == 50:
                #     break
            full_season_df = pd.concat(pieces)
            full_season_df.to_csv(f'{self.data_path}/bbref-files/{season}.csv', index = False)

    def get_table_info(self, soup, link):
        columns = ['Player', 'Date', 'Team', 'Against', 'Home', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
                   'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-']
        # column_2 = ['TS%', 'eFG%',
        #            'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']
        header = soup.findAll('h1')[0].text.split(',')
        date = ''.join(header[-2:]).strip()
        teams = header[0].split('at')
        # away_team = teams[0].strip()
        # home_team = teams[1][:teams[1].find('Box Score')].strip()
        away_team = teams[0].strip()
        home_team = teams[-1][:teams[-1].find('Box Score')].strip()
        if home_team == '':
            home_team = teams[-2][:teams[-2].find('Box Score')].strip()
        tables = soup.findAll('tbody')
        #get unqiue players
        player_dict = {}
        away_idx = [0]
        home_idx = [8]
        for idx, table in enumerate(tables):
            # print(idx)
            # print(table)
            # print('~'*50)
            if idx not in away_idx + home_idx:
                continue
            if idx in away_idx:
                team = away_team
                opp = home_team
                home = 0
            else:
                team = home_team
                opp = away_team
                home = 1
            for row in table:
                try:
                    name = row.findAll('th')[0].text
                    if name == 'Reserves':
                        continue
                    player_dict[name] = [name, date, team, opp, home]
                except AttributeError:
                    continue
        for idx, table in enumerate(tables):
            if idx not in away_idx + home_idx:
                continue
            for row in table:
                try:
                    name = row.findAll('th')[0].text
                    cols = row.findAll('td')
                    if len(cols) == 0:
                        continue
                    cols = [i.text.strip() for i in cols]
                    for c in cols:
                        player_dict[name].append(c)
                except AttributeError:
                    continue
        game_df_pieces = []
        for name, l in player_dict.items():
            row_dict = {col:value for col, value in zip(columns, l)}
            row_df = pd.DataFrame(row_dict, index = [0])
            game_df_pieces.append(row_df)
        game_df = pd.concat(game_df_pieces)
        game_df['GameLink'] = [link for i in range(len(game_df))]
        # game_df.to_csv('Tester.csv', index = False)
        # game_df.to_csv('Tester.csv', index = False)
        return game_df

    def get_game_links(self):
        self.full_game_urls = {}
        pbar = tqdm((enumerate(self.month_url_dict.items())), total = len(self.month_url_dict))
        for idx, (season, month_url_list) in pbar:
            pbar.set_description(f'({idx+1}-{len(self.month_url_dict)}) | Getting Game URLs: {season}')
            season_game_urls = []
            for month_url in month_url_list:
                response = requests.get(month_url)
                soup = BeautifulSoup(response.text, 'html.parser')
                time.sleep(1)
                table = soup.findAll('tbody')
                # print(table)
                game_links = table[0].findAll('a', href = True)
                for idx, link in enumerate(game_links):
                    if (idx + 1) % 4 != 0:
                        continue
                    if link.text.strip() != 'Box Score':
                        continue
                    full_url = f'{self.base_url}{link["href"]}'
                    season_game_urls.append(full_url)

            self.full_game_urls[season] = season_game_urls
        pbar.close()
        # pickle.dump(self.full_game_urls, open(f'{self.data_path}/pickles/GameLinks.p', 'wb'))
        del self.month_url_dict

    def get_months(self):
        if self.scrape_type == 0:
            months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
        elif self.scrape_type == 1:
            months = ['november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
        elif self.scrape_type == 2:
            months = ['december', 'january', 'february', 'march', 'april', 'may', 'june']
        elif self.scrape_type == 3:
            months = ['october-2019', 'november', 'december', 'january', 'february', 'march', 'july', 'august', 'september', 'october-2020']
        elif self.scrape_type == 4:
            months = ['december', 'january', 'february', 'march']
        self.month_url_dict = {}
        for season_start_link in self.season_start_links:
            response = requests.get(season_start_link)
            soup = BeautifulSoup(response.text, 'html.parser')
            season = soup.find('h1').text.split(' ')[0].strip()
            if os.path.exists(f'{self.data_path}/bbref-files/{season}.csv'):
                continue
            year = f'20{season[season.find("-")+1:]}'
            season_month_list = []
            for month in months:
                base_url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
                season_month_list.append(base_url)
            self.month_url_dict[season] = season_month_list
        # print(self.month_url_dict[season])

if __name__ == '__main__':

    season_list_3 = [
        'https://www.basketball-reference.com/leagues/NBA_2020_games.html'
    ]
    season_list_2 = [
        'https://www.basketball-reference.com/leagues/NBA_2012_games.html',
    ]
    season_list_1 = [
        'https://www.basketball-reference.com/leagues/NBA_2005_games.html',
        'https://www.basketball-reference.com/leagues/NBA_2006_games.html',
    ]       #doesnt work with current method

    season_list_0 = [
        #'https://www.basketball-reference.com/leagues/NBA_2001_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2002_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2003_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2004_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2007_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2008_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2009_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2010_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2011_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2013_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2014_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2015_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2016_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2017_games.html',
        #'https://www.basketball-reference.com/leagues/NBA_2018_games.html',
        'https://www.basketball-reference.com/leagues/NBA_2019_games.html',
        'https://www.basketball-reference.com/leagues/NBA_2020_games.html',
        'https://www.basketball-reference.com/leagues/NBA_2021_games.html'

    ]
    season_dict = {0: season_list_0, 1: season_list_1, 2: season_list_2, 3: season_list_3}
    total = len(season_list_0) + len(season_list_1) + len(season_list_2) + len(season_list_3)
    # print(f'Total Games: {total}')
    # for scrape_type, season_list in season_dict.items():
        # nba_stat_scraper = BbrefScraper(season_list, scrape_type = scrape_type)
        # nba_stat_scraper.scrape_stats()

    season_list = [
       'https://www.basketball-reference.com/leagues/NBA_2021_games.html'
    ]

    nba_stat_scraper = BbrefScraper(season_list_0, scrape_type = 4)
    nba_stat_scraper.scrape_stats()

Scraping
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
 https://www.basketball-reference.com/leagues/NBA_2019_games.html
https://www.basketball-reference.com/leagues/NBA_2020_games.html
https://www.basketball-reference.com/leagues/NBA_2021_games.html


(3-3) | Getting Game URLs: 2020-21: 100%|██████████| 3/3 [00:15<00:00,  5.07s/it]
Scraping Games: 2020-21: 100%|██████████| 705/705 [05:46<00:00,  2.03it/s]


# Clean the data:

In [51]:
import pandas as pd
import os
import shutil
import numpy as np


In [52]:

data_dir = '/Users/yoavhaim/Desktop/Programming/Project/bbref-files'
cleaned_data_dir = '/Users/yoavhaim/Desktop/Programming/Project/cleaned'
ignore = ['cleaned','.DS_Store']
file_paths = [i for i in os.listdir(data_dir) if i not in ignore]
file_paths

['2019-20.csv', '2018-19.csv', '2020-21.csv']

In [53]:
team_names = []
for csv_path in file_paths:
    print(csv_path)
    csv_path = f'{data_dir}/{csv_path}'
    df = pd.read_csv(csv_path)
    team_names += df.Team.unique().tolist()
    team_names += df.Against.unique().tolist()


2019-20.csv
2018-19.csv
2020-21.csv


In [54]:
print(set(team_names))


{'Toronto Raptors', 'Golden St', 'Los Angeles Clippers', 'Denver Nuggets', 'Philadelphia 76ers', 'Oklahoma City Thunder', 'Chicago Bulls', 'Los Angeles Lakers', 'Indiana Pacers', 'Miami He', 'Minnesota Timberwolves', 'Washington Wizards', 'New York Knicks', 'e Warriors', 'Utah Jazz', 'Detroit Pistons', 'Milwaukee Bucks', 'Orlando Magic', 'Sacramento Kings', 'Atlanta Hawks', 'Dallas Mavericks', 'Portland Trail Blazers', 'Charlotte Hornets', 'Miami H', 'Houston Rockets', 'Phoenix Suns', 'Brooklyn Nets', 'New Orleans Pelicans', 'San Antonio Spurs', 'Boston Celtics', 'Memphis Grizzlies', 'Cleveland Cavaliers'}


In [55]:
def convert_teams(x):
    if x in ['Se', 'tle SuperSonics']:
        return 'Seattle SuperSonics'
    elif x in ['Golden St', 'e Warriors']: 
        return 'Golden State Warriors'
    elif x in ['Miami H', 'Miami He']: 
        return 'Miami Heat'
    elif x in ['s']:
        return 'Charlotte Bobcats'
    else: 
        return x
def convert_mp(x):
    if type(x) == float: 
        return None
    elif x in ['Did Not Play', 'Did Not Dress', 'Not With Team', 'Player Suspended']:
        return None
    else:
        x = x.split(':')
        x = float('.'.join(x))
        return x
def convert_fg(x):
    if type(x) == float:
        return None
    elif x in ['Did Not Play', 'Did Not Dress', 'Not With Team', 'Player Suspended']:
        return None
    else:
        return int(x)
columns = ['Player', 'Date', 'Team', 'Against', 'Home', 'MP', 'FG', 'FGA', 
           'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
           'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-', 'GameLink']
df_team_cleaned = {}
for csv_path in file_paths:
    new_path = f'{data_dir}/{csv_path}'
    df = pd.read_csv(new_path)
    df['Team'] = df.Team.map(convert_teams)
    df['Against'] = df.Against.map(convert_teams)
    df['Date'] = pd.to_datetime(df.Date)
    df['MP'] = df.MP.map(convert_mp)
    df['FG'] = df.FG.map(convert_fg)
    df_team_cleaned[csv_path] = df
    df.to_csv(f'{cleaned_data_dir}/{csv_path}', index = False)


## Removing missing rows & adding additional columns:

In [56]:
data_d = '/Users/yoavhaim/Desktop/Programming/Project/cleaned'

csv_paths = [i for i in os.listdir(data_d) if i not in ignore]


In [57]:
# cleaned_df_dict

cleaned_directory = '/Users/yoavhaim/Desktop/Programming/Project/cleaned'
csv_dict = {i: pd.read_csv(f'{cleaned_directory}/{i}') for i in os.listdir(cleaned_directory) if 'Combined' not in i}


### Removing empty rows:

In [87]:
droppedmissing_dir='/Users/yoavhaim/Desktop/Programming/Project/Dropped'
def droppedmissing(df_dict):
    new_df_dict = {}
    for key, df in df_dict.items():
        directory= f'{cleaned_directory}/{key}'
        df = pd.read_csv(directory)
        df=df.dropna()
        df.to_csv(f'{droppedmissing_dir}/{key}', index = False)
    return new_df_dict

cleaned_df_dict = droppedmissing(csv_dict)

In [88]:
data_d = '/Users/yoavhaim/Desktop/Programming/Project/Dropped'
ignore = ['cleaned','.DS_Store']

csv_paths = [i for i in os.listdir(data_d) if i not in ignore]
# dropped_df_dict
ignore = ['cleaned','.DS_Store']
dropped_directory = '/Users/yoavhaim/Desktop/Programming/Project/Dropped'

csv_dict = {i: pd.read_csv(f'{data_d}/{i}') for i in os.listdir(data_d) if i not in ignore}


### Adding special veriables

In [89]:
Changed_directory = '/Users/yoavhaim/Desktop/Programming/Project/Changed'
def add_variables(df_dict):
    new_df_dict = {}
    for key, df in df_dict.items():
        directory= f'{dropped_directory}/{key}'
        df = pd.read_csv(directory)
        #adding TS%

        df['TS%']=df['PTS']/2*(df['FGA']+0.44*(df['FTA']))
        #calculating DD and DT
        mask=(((df['TRB']>9) & (df['PTS']>9)) | ((df['PTS']>9) & (df['AST']>9)) | ((df['TRB']>9) & (df['AST']>9)))
        df['DD']=1*mask
        mask=(((df['TRB']>9) & (df['PTS']>9) & (df['AST']>9)) | ((df['PTS']>9) & (df['AST']>9) & (df['TRB']>9)))
        df['DT']=1*mask
        #calculating FPTS
        #based on this- https://www.draftkings.com/help/rules/4/70
        df['FPTS']= 1*df['PTS']+0.5*df['3P']+1.25*df['TRB']+1.5*df['AST']+2*df['STL']+2*df['BLK']-0.5*df['TOV']+1*df['DD']+1*df['DT']
        #new_df_dict[key] = df
        df.to_csv(f'{Changed_directory}/{key}', index = False)
    return new_df_dict

cleaned_df_dict = add_variables(csv_dict)

# Join DF score dictionary

In [127]:

def add_df_score(filename1, filename2,year):
    df1 = pd.read_csv(filename1)
    df2 = pd.read_csv(filename2)
    df2['Against']=df2['Full Name']
    df2['D_R']=df2[f'{year}']
    df2=df2.drop(['Full Name','2019-2020','2020-2021','2018-2019'], axis=1)
    df1=df1.merge(df2, left_on='Against', right_on='Against')
    df1.to_csv(f'/Users/yoavhaim/Desktop/Programming/Project/with DF rating/{year}.csv', index = False)
    return
#notice to chose the year based on the file you want:
year='2020-21'
filename1_dir=(f'/Users/yoavhaim/Desktop/Programming/Project/Changed/{year}.csv'
filename2_dir='/Users/yoavhaim/Desktop/Programming/Project/Changed/Defensive Ratings.csv'
add_df_score(filename1_dir, filename2_dir,year)

# Converting the data to players:

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
import numpy as np
import pickle
from tqdm import  tqdm
import shutil

pd.options.mode.chained_assignment = None
pd.options.display.min_rows = 50

In [145]:
#CREATE THE COMBINED DF AND SPLIT PLAYERS OUT TO THEIR OWN FILES

bbreff_dir = '/Users/yoavhaim/Desktop/Programming/Project/Changed'

pieces = []
for csv_name in os.listdir(bbreff_dir):
    full_path = f'{bbreff_dir}/{csv_name}'
    season = csv_name[:csv_name.find('.csv')]
    df = pd.read_csv(full_path)
    df['season'] = [season for _ in range(len(df))]
    pieces.append(df)

full_df = pd.concat(pieces)


original_player_dir = '/Users/yoavhaim/Desktop/Programming/Project/players/data'
pbar = tqdm(full_df.Player.unique(), desc = 'Getting CSV for each player..')
for player in pbar:
    subset_df = full_df[full_df.Player == player]
    subset_df.to_csv(f'{original_player_dir}/{player}.csv', index = False)
pbar.close()


Getting CSV for each player..: 100%|██████████| 618/618 [00:02<00:00, 269.89it/s]


# Creating a shift in the data:

In [128]:
changed_d = '/Users/yoavhaim/Desktop/Programming/Project/with DF rating'
ignore = ['cleaned','.DS_Store']
csv_dict = {i: pd.read_csv(f'{changed_d}/{i}') for i in os.listdir(changed_d)  if i not in ignore}
csv_dict

{'2019-20.csv':                        Player        Date                Team  \
 0                Jimmy Butler  2019-12-01          Miami Heat   
 1             Duncan Robinson  2019-12-01          Miami Heat   
 2                Goran Dragić  2019-12-01          Miami Heat   
 3             Justise Winslow  2019-12-01          Miami Heat   
 4                  Trae Young  2019-12-04       Atlanta Hawks   
 5                 Cam Reddish  2019-12-04       Atlanta Hawks   
 6                Terry Rozier  2019-12-06   Charlotte Hornets   
 7             Devonte' Graham  2019-12-06   Charlotte Hornets   
 8               Miles Bridges  2019-12-06   Charlotte Hornets   
 9                 Cody Zeller  2019-12-06   Charlotte Hornets   
 10            Marvin Williams  2019-12-06   Charlotte Hornets   
 11               Nikola Jokić  2019-12-08      Denver Nuggets   
 12               Jamal Murray  2019-12-08      Denver Nuggets   
 13                Will Barton  2019-12-08      Denver Nugget

In [129]:

def create_full_shift(df_dict, columns, timestep):
    df_pieces = []
    for key, df in df_dict.items():
        season = key[:key.find('.csv')]
        df.Date = pd.to_datetime(df.Date)
        pieces = []
        unique_players = df.Player.unique()
        pbar = tqdm(unique_players)
        player_pieces = []
        for player in pbar: 
            pbar.set_description(f'{key}: Shifting the DF w/ timestep {timestep}: {player}')
            player_df = df[df.Player == player]
            player_df = player_df.sort_values(by = 'Date', ascending = True)
            for column in columns:
#                 player_df[column] = StandardScaler().fit_transform(player_df[column].values.reshape(-1,1))
                for step in range(1, timestep + 1):
                    player_df[f'{column}_{step}'] = player_df[column].shift(step)
            player_pieces.append(player_df)
        final_df = pd.concat(player_pieces)
        final_df['Season'] = [season for i in range(len(final_df))]
        df_pieces.append(final_df)
    full_final_shifted = pd.concat(df_pieces)
    full_final_shifted.dropna(inplace = True)
    full_final_shifted.to_csv(f'{changed_d}/Combined_{timestep}.csv', index = False)
      
    return full_final_shifted

# test_df = create_full_shift(cleaned_df_dict, ['FDP'], 5)
# test_df
shutil.rmtree(changed_d)
os.mkdir(changed_d)
df_dict = {}
for timestep in range(1, 21):       
    df_dict[timestep] = create_full_shift(csv_dict, ['FPTS', 'PTS','AST','STL','BLK', 'DD','DT', 'MP','FGA' ], timestep)

2019-20.csv: Shifting the DF w/ timestep 1: Deandre Ayton: 100%|██████████| 427/427 [00:03<00:00, 132.23it/s]           
2018-19.csv: Shifting the DF w/ timestep 1: Justin Patton: 100%|██████████| 432/432 [00:03<00:00, 141.49it/s]           
2020-21.csv: Shifting the DF w/ timestep 1: Sindarius Thornwell: 100%|██████████| 433/433 [00:03<00:00, 133.02it/s]     
2019-20.csv: Shifting the DF w/ timestep 2: Deandre Ayton: 100%|██████████| 427/427 [00:04<00:00, 86.83it/s]           
2018-19.csv: Shifting the DF w/ timestep 2: Justin Patton: 100%|██████████| 432/432 [00:04<00:00, 96.24it/s]           
2020-21.csv: Shifting the DF w/ timestep 2: Sindarius Thornwell: 100%|██████████| 433/433 [00:04<00:00, 97.97it/s]     
2019-20.csv: Shifting the DF w/ timestep 3: Deandre Ayton: 100%|██████████| 427/427 [00:06<00:00, 70.48it/s]           
2018-19.csv: Shifting the DF w/ timestep 3: Justin Patton: 100%|██████████| 432/432 [00:06<00:00, 71.33it/s]           
2020-21.csv: Shifting the DF w/ times

# Create moving averages for the 10 day shift

In [130]:
Changed_directory = '/Users/yoavhaim/Desktop/Programming/Project/with DF rating'
def create_moving_avg(directory, number):
    directory= f'{Changed_directory}/Combined_{number}.csv'
    df = pd.read_csv(directory)
    columns= df.columns
    FPTS_index= df.columns.get_loc("FPTS")
    df[f'FPTS{number}_MA']= df.iloc[:, FPTS_index+1:FPTS_index+10].mean(axis=1)
    PTS_index= df.columns.get_loc("PTS")
    df[f'PTS{number}_MA']= df.iloc[:, PTS_index+1:PTS_index+10].mean(axis=1)
    AST_index= df.columns.get_loc("AST")
    df[f'AST{number}_MA']= df.iloc[:, AST_index+1:AST_index+10].mean(axis=1)
    STL_index= df.columns.get_loc("STL")
    df[f'STL{number}_MA']= df.iloc[:, STL_index+1:STL_index+10].mean(axis=1)
    BLK_index= df.columns.get_loc("BLK")
    df[f'BLK{number}_MA']= df.iloc[:, BLK_index+1:STL_index+10].mean(axis=1)
    DD_index= df.columns.get_loc("DD")
    df[f'DD{number}_MA']= df.iloc[:, BLK_index+1:DD_index+10].mean(axis=1)
    DT_index= df.columns.get_loc("DT")
    df[f'DT{number}_MA']= df.iloc[:, DT_index+1:DT_index+10].mean(axis=1)
    MP_index= df.columns.get_loc("MP")
    df[f'MP{number}_MA']= df.iloc[:, MP_index+1:MP_index+10].mean(axis=1)
    FGA_index= df.columns.get_loc("FGA")
    df[f'FGA{number}_MA']= df.iloc[:, FGA_index+1:FGA_index+10].mean(axis=1)
    df.to_csv(f'{Changed_directory}/ma{number}.csv', index = False)
    
    return 

create_moving_avg(Changed_directory,10)


In [37]:
# cleaned_df_dict

players_d = '/Users/yoavhaim/Desktop/Programming/Project/changed'
ignore = ['cleaned','.DS_Store']
csv_dict = {i: pd.read_csv(f'{players_d}/{i}') for i in os.listdir(players_d)  if i not in ignore}
csv_dict

{'merged.csv':                          Player     Date                   Team  \
 0         Giannis Antetokounmpo  12/1/18        New York Knicks   
 1                  Eric Bledsoe  12/1/18        New York Knicks   
 2                   Brook Lopez  12/1/18        New York Knicks   
 3               Khris Middleton  12/1/18        New York Knicks   
 4               Pat Connaughton  12/1/18        New York Knicks   
 5                Sterling Brown  12/1/18        New York Knicks   
 6                    Tony Snell  12/1/18        New York Knicks   
 7                    Thon Maker  12/1/18        New York Knicks   
 8                  Kevin Durant  12/1/18  Golden State Warriors   
 9                 Stephen Curry  12/1/18  Golden State Warriors   
 10               Andre Iguodala  12/1/18  Golden State Warriors   
 11                Blake Griffin  12/1/18        Detroit Pistons   
 12               Andre Drummond  12/1/18        Detroit Pistons   
 13               Reggie Jackson  

## Now for some Machine learning:

In [146]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import pickle
import os
import shutil
import matplotlib.pyplot as plt


In [18]:
#creating the test and train set
merged_dir='/Users/yoavhaim/Desktop/Programming/Project/Changed/merged.csv'
df=pd.read_csv(merged_dir)
df = df.sample(frac=.3,random_state=42)
from sklearn.model_selection import train_test_split
y=df.FPTS
x=df.drop('FPTS',axis=1)
x=x.drop(['GameLink','date,axis=1)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.head()

,Player,Date,Team,Against,Home,MP,FG,FGA,FG%,3P,...,STL,BLK,TOV,PF,PTS,+/-,GameLink,TS%,DD,DT
12506,Cameron Johnson,2/8/20,Phoenix Suns,Denver Nuggets,1,23.34,3,8,0.375,2,...,1,1,0,0,11,5,https://www.basketball-reference.com/boxscores...,51.26,0,0
4205,Monte Morris,2/1/19,Denver Nuggets,Houston Rockets,1,36.53,7,11,0.636,3,...,2,0,1,1,18,21,https://www.basketball-reference.com/boxscores...,106.92,0,0
13410,Kelan Martin,2/28/20,Minnesota Timberwolves,Orlando Magic,0,11.41,1,4,0.250,0,...,1,0,1,4,4,-18,https://www.basketball-reference.com/boxscores...,9.76,0,0
18463,Nemanja Bjelica,2/20/21,Sacramento Kings,Chicago Bulls,0,11.52,3,7,0.429,1,...,0,0,0,0,9,-3,https://www.basketball-reference.com/boxscores...,35.46,0,0
9242,Fred VanVleet,12/22/19,Toronto Raptors,Dallas Mavericks,1,27.35,2,13,0.154,1,...,1,0,4,1,10,-20,https://www.basketball-reference.com/boxscores...,78.20,0,0


In [ ]:

def get_train_split(timestep):

    pickle_dir = '/home/samuel-linux/PycharmProjects/Personal/FantasyBasketball/Data/pickles/roto/shifted'
    split_dict = pickle.load(open(f'{pickle_dir}/{timestep}-TrainTestSplit.p', 'rb'))
        
    x_train, y_train = split_dict['x_train'], split_dict['y_train'] 
    x_test, y_test = split_dict['x_test'], split_dict['y_test']
    x_val, y_val  = split_dict['x_val'], split_dict['y_val']
    
    columns = [i for i in x_train.columns if i != 'Player']
    x_train = x_train[columns]
    x_test = x_test[columns]
    x_val = x_val[columns]
    return x_train, y_train, x_test, y_test, x_val, y_val

x_train, y_train, x_test, y_test, x_val, y_val = get_train_split(3)


<h4>reset the index of X_train and y_train to 0...n</h4>